# Getting started: chapter 10

## 10. Enumerables and streams

> Elixir provides the concept of enumerables and [the Enum module](https://hexdocs.pm/elixir/Enum.html) to work with them. We have already learned two enumerables: lists and maps.

In [1]:
Enum.map([1, 2, 3], fn x -> x * 2 end)

[2, 4, 6]

In [2]:
Enum.map(%{1 => 2, 3 => 4}, fn {k, v} -> k * v end)

[2, 12]

### Ranges

In [3]:
Enum.reduce(1..4, 0, &+/2)

10

In [4]:
Enum.map(1..4, fn x -> x * 2 end)

[2, 4, 6, 8]

Functions in the `Enum` module can work with any data type that implements the [Enumerable protocol](https://hexdocs.pm/elixir/Enumerable.html).

### Eager vs. lazy

In [5]:
odd? = &(rem(&1, 2) != 0)
Enum.filter(1..10, odd?)

[1, 3, 5, 7, 9]

In [6]:
total_sum = Enum.sum(Enum.filter(Enum.map(1..100_000, &(&1 * 3)), odd?))

7500000000

The [pipe operator](https://hexdocs.pm/elixir/Kernel.html#%7C%3E/2):

In [7]:
total_sum = 1..100_000 |> Enum.map(&(&1 * 3)) |> Enum.filter(odd?) |> Enum.sum

7500000000

### Streams

In [8]:
1..100_000 |> Stream.map(&(&1 * 3)) |> Stream.filter(odd?) |> Enum.sum

7500000000

In [9]:
1..100_000 |> Stream.map(&(&1 * 3))

#Stream<[enum: 1..100000, funs: [#Function<49.126435914/1 in Stream.map/2>]]>

In [10]:
stream = Stream.cycle(1..3)

#Function<10.126435914/2 in Stream.cycle/1>

In [11]:
stream |> Enum.take(10)

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1]

In [12]:
stream = Stream.unfold("façade", &String.next_codepoint/1)

#Function<65.126435914/2 in Stream.unfold/2>

In [13]:
Enum.take(stream, 3)

["f", "a", "ç"]

In [14]:
# data source: https://www.unicode.org/Public/UCD/latest/ucd/UnicodeData.txt
stream = File.stream!("/Users/luciano/UnicodeData.txt")

%File.Stream{line_or_bytes: :line, modes: [:raw, :read_ahead, :binary], path: "/Users/luciano/UnicodeData.txt", raw: true}

In [15]:
sample_line = "0041;LATIN CAPITAL LETTER A;Lu;0;L;;;;;N;;;;0061;"

format_line = fn line ->
  [code, name | _] = String.split(line, ";")
  char = <<String.to_integer(code, 16)::utf8>>
  "U+#{code}\t#{char}\t#{name}"
end

format_line.(sample_line) |> IO.puts

U+0041	A	LATIN CAPITAL LETTER A


:ok

In [16]:
# data source: https://www.unicode.org/Public/UCD/latest/ucd/UnicodeData.txt
File.stream!("/Users/luciano/UnicodeData.txt")
|> Stream.drop(32)
|> Stream.take(20)
|> Stream.map(format_line)
|> Enum.each(&IO.puts(&1))

U+0020	 	SPACE
U+0021	!	EXCLAMATION MARK
U+0022	"	QUOTATION MARK
U+0023	#	NUMBER SIGN
U+0024	$	DOLLAR SIGN
U+0025	%	PERCENT SIGN
U+0026	&	AMPERSAND
U+0027	'	APOSTROPHE
U+0028	(	LEFT PARENTHESIS
U+0029	)	RIGHT PARENTHESIS
U+002A	*	ASTERISK
U+002B	+	PLUS SIGN
U+002C	,	COMMA
U+002D	-	HYPHEN-MINUS
U+002E	.	FULL STOP
U+002F	/	SOLIDUS
U+0030	0	DIGIT ZERO
U+0031	1	DIGIT ONE
U+0032	2	DIGIT TWO
U+0033	3	DIGIT THREE


:ok